In [15]:
import requests
from bs4 import BeautifulSoup
import time
import threading
import urllib.request #Save Image
import webbrowser #멮쥐
#File
from datetime import datetime
import os
from PIL import ImageTk
import PIL.Image
from tkinter import filedialog
#UI
from tkinter import *
from tkinter import ttk
import tkinter.messagebox

In [20]:
class HTML():
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    
    def GetHTML(url):
        response = requests.get(url, headers=HTML.headers) # 세션으로 하면 더 오래 할 수 있던가?
        # ip차단 방지를 위해 일시정지
        time.sleep(0.9)
        return BeautifulSoup(response.text, "html.parser")
    
    def GetUnionTop(html):
        try:
            data_list = html.select('table.rank_table > tbody > tr > td')
            return data_list[2].text.replace(',', '')
        except(IndexError):
            return None

    def GetUnionBottom(html):
        try:
            data_list = html.select('table.rank_table > tbody > tr > td')
            return data_list[38].text.replace(',', '')
        except(IndexError):
            return None
        
    def GetCharacterInfoAll(html):
        # 모든 캐릭터 정보를 저장할 리스트 선언
        data_tuple_list = []

        # 모든 캐릭터 정보 추출
        character_info_list = html.select('table.rank_table > tbody > tr')

        for character_info in character_info_list:
            nick = character_info.select_one('a').text
            job = character_info.select_one('dd').text.replace(',', '')
            union = character_info.select('td')[2].text.replace(',', '')
            img_url = character_info.select_one('span.char_img > img')["src"]
            data_tuple_list.append((nick, job, union, img_url))

        return data_tuple_list

In [17]:
class Crawlling(HTML):
    url = None
    search_min = 1
    search_max = 1
    
    def TempCode_InputInfo():
        GUI.server_combobox.set("크로아")
        GUI.union_min_entry.delete(0, len(GUI.union_min_entry.get()))
        GUI.union_max_entry.delete(0, len(GUI.union_max_entry.get()))
        GUI.union_min_entry.insert(0, '9055')
        GUI.union_max_entry.insert(0, '9055')
    
    def ValidateCrawlling_Thread():
        t1=threading.Thread(target=Crawlling.ValidateCrawlling)
        t1.setDaemon(True)
        t1.deamon = True
        t1.start()

    def ValidateCrawlling():
        # 테스트용 자동 입력
#         Crawlling.TempCode_InputInfo()
        
        # 입력 오류 체크
        Crawlling.IsExistInputError()
        
        # 서버에 따른 URL 설정
        Crawlling.SetURL()
        
        # 탐색 범위 설정(1000단위, 100단위, 10단위, 5단위로)
        Crawlling.SetSearchRange(1, 100000, 1000)
        Crawlling.SetSearchRange(Crawlling.search_min, Crawlling.search_max+100, 100)
        Crawlling.SetSearchRange(Crawlling.search_min, Crawlling.search_max+10, 10)
        Crawlling.SetSearchRange(Crawlling.search_min, Crawlling.search_max+5, 5)
        GUI.SetProgressLabelMessage(f"검색 범위 설정 완료 [{Crawlling.search_min} ~ {Crawlling.search_max}]", 75)
        
        # 조건에 맞는 캐릭터 탐색
        Crawlling.FindSuspect()
        
        # 파일 저장 
        CsvFile.SaveCsvFile()
        
        # 결과 출력
        
    def IsExistInputError():
        pass
    
    def SetURL():
        server = GUI.GetServer()
        if server == '리부트2': server_code = 'w=1'
        elif server == '리부트1': server_code = 'w=2'
        elif server == '오로라': server_code = 'w=3'
        elif server == '레드': server_code = 'w=4'
        elif server == '이노시스': server_code = 'w=5'
        elif server == '유니온': server_code = 'w=6'
        elif server == '스카니아': server_code = 'w=7'
        elif server == '루나': server_code = 'w=8'
        elif server == '제니스': server_code = 'w=9'
        elif server == '크로아': server_code = 'w=10'
        elif server == '베라': server_code = 'w=11'
        elif server == '엘리시움': server_code = 'w=12'
        elif server == '아케인': server_code = 'w=13'
        elif server == '노바': server_code = 'w=14'
        Crawlling.url = f"https://maplestory.nexon.com/Ranking/Union?{server_code}&page="
        
    def SetSearchRange(start, end, increase):
        union_range_min = int(GUI.GetUnionMin())
        union_range_max = int(GUI.GetUnionMax())

        # 검색 범위 설정
        for i in range(start, end, increase):
            html = Crawlling.GetHTML(f"{Crawlling.url}{i}")
            union_top = int(Crawlling.GetUnionTop(html))
            union_bottom = int(Crawlling.GetUnionBottom(html))
            if union_top == None or union_bottom == None:
                Crawlling.search_max = i
                break
            if union_top > union_range_max:
                Crawlling.search_min = i
            if union_bottom < union_range_min:
                Crawlling.search_max = i
                break
            GUI.SetProgressLabelMessage(f"검색 범위를 설정 중입니다... [{i} ~ {Crawlling.search_max+increase}]", 60)
        
    def FindSuspect():
        union_range_min = int(GUI.GetUnionMin())
        union_range_max = int(GUI.GetUnionMax())
        
        for i in range(Crawlling.search_min, Crawlling.search_max+1):
            html = Crawlling.GetHTML(f"{Crawlling.url}{i}")
            data_tuple_list = Crawlling.GetCharacterInfoAll(html)
            
            # 조건 일치 시 CsvFile 텍스트 내 저장 
            for data_tuple in data_tuple_list:
                character_union = int(data_tuple[2])
                if character_union >= union_range_min and character_union <= union_range_max:
                    nick = data_tuple[0]
                    job = data_tuple[1]
                    img_url = data_tuple[3]
                    CsvFile.AppendCsvText(f"{nick},{job},{character_union}{','*47}{img_url}")
            GUI.SetProgressLabelMessage(f"조건에 맞는 캐릭터 탐색 중입니다... [{i} / {Crawlling.search_max}]", 50)
            
    
    def ShowResult():
        pass

In [18]:
class CsvFile():
    csv_text = "닉네임,직업,유니온\n"
    
    def ClearCsvText():
        CsvFile.csv_text = "닉네임,직업,유니온\n"
        
    def AppendCsvText(text):
        CsvFile.csv_text += f"{text}\n"
        
    def SaveCsvFile():
        GUI.SetProgressLabelMessage("결과 파일을 저장 중 입니다...", 85)
        file_name = f"[{GUI.GetServer()}] {GUI.GetUnionMin()} ~ {GUI.GetUnionMax()} 결과.csv"
        with open(file_name,'w') as file:
            file.write(CsvFile.csv_text)
        tkinter.messagebox.showinfo("Result", f"{file_name}\n 파일이 저장되었습니다.")
        GUI.SetProgressLabelMessage("", 85)

In [19]:
class UiFunction():
    def SetProgressLabelMessage(self, text, x):
        self.progress_label.config(text = text)
        self.progress_label.place(x = x, y = 230)
        
    def GetServer(self):
        return self.server_combobox.get()
        
    def GetUnionMin(self):
        return self.union_min_entry.get()
        
    def GetUnionMax(self):
        return self.union_max_entry.get()

class Ui(UiFunction):
    def __init__(self):
        self.win = Tk()
        self.win.title(f'추노 - 본캐 추적 프로그램')    
        self.win.geometry('330x300')
        self.win.option_add('*Font', '맑은고딕 10')
        self.win.resizable(False, False)
        
        #Default Ui
        Ui.MenuBar(self)
        Ui.MainImage(self)
        Ui.MainUi(self)

    def MenuBar(self):
        def GoToMyBlog():
            webbrowser.open("https://blog.naver.com/PostList.naver?blogId=wkrnjs456&categoryNo=8&skinType=&skinId=&from=menu")
            
        def GoToMyGithub():
            webbrowser.open("https://github.com/JaGwonLim")
        self.menubar = Menu(self.win)
        self.menu_1 = Menu(self.menubar, tearoff=0)
        self.menu_1.add_command(label="Help", command = GoToMyBlog)
        self.menu_1.add_command(label="Github", command = GoToMyGithub)
        self.menubar.add_cascade(label="Option", menu=self.menu_1)
        self.win.config(menu=self.menubar)
        
    def MainImage(self):
        try:
            self.label_image = Label(self.win)
            main_image_url = "https://postfiles.pstatic.net/MjAyMjAxMjZfNzAg/MDAxNjQzMTc0NjkzNTYw.2DrFjsCBdwiB8m-HWXkHSPp8qKvjtMISOFbqgw9egaAg.gO_uTBQ7hCS_8k8INmH8x_awRzGyEpZq57atEpw4b5Mg.PNG.wkrnjs456/%EC%B6%94%EB%85%B8-%EB%B0%98.png?type=w966"
            self.main_image = ImageTk.PhotoImage(PIL.Image.open(requests.get(main_image_url, stream=True).raw).resize((350,70)))
            self.label_image = Label(self.win, image = self.main_image)
            self.label_image.pack()
        except:
            #사진 삭제 등으로 인한 오류 방지
            pass
    
    def MainUi(self):        
        # Server UI
        self.server_label = Label(self.win, text = '[서버 설정]')

        self.server_list =['루나', '스카니아', '엘리시움', '크로아', '베라', 
                      '오로라', '레드', '제니스', '이노시스', '유니온', 
                      '아케인', '노바', '리부트1', '리부트2'] 

        self.server_combobox = ttk.Combobox(self.win, 
                                       width = 11, 
                                       values = self.server_list, 
                                       state="readonly")
        self.server_combobox.set("서버 선택")
        
        self.server_label.place(x = 35 , y = 130)
        self.server_combobox.place(x = 140, y = 130) 
        
        # Union Range Ui
        self.union_label = Label(self.win, text = '[유니온 범위]')
        self.union_min_entry = Entry(self.win, width = 5)
        self.union_range_label = Label(self.win, text = '~')
        self.union_max_entry = Entry(self.win, width = 5)
        
        self.union_label.place(x = 30, y = 180)
        self.union_min_entry.place(x = 140, y = 180)
        self.union_range_label.place(x = 182, y = 180)
        self.union_max_entry.place(x = 200, y = 180)
        
        # Progress Ui
        self.progress_label = Label(self.win, text = '', font = "맑은고딕 8")
        self.progress_label.place(x = 40, y = 230)
        
        # Activate Button
        self.activate_button = Button(self.win, text = "Start", width = 25, height = 2, command = Crawlling.ValidateCrawlling_Thread)
        self.activate_button.place(x = 55, y = 250)
            
GUI = Ui()
GUI.win.mainloop()

In [110]:
self.SetProgressLabelMessage('12.0% [ 12 / 100 ] (18.25s)', 90)

NameError: name 'self' is not defined

In [40]:
# 테스트용
html = Crawlling.GetHTML('https://maplestory.nexon.com/Ranking/Union?page=9881&w=10')

def GetUnionTop(html):
    try:
        data_list = html.select('table.rank_table > tbody > tr > td')
        return data_list[2].text.replace(',', '')
    except(IndexError):
        return None
    
def GetUnionBottom(html):
    try:
        data_list = html.select('table.rank_table > tbody > tr > td')
        return data_list[38].text.replace(',', '')
    except(IndexError):
        return None
    
def GetCharacterInfoAll(html):
    # 모든 캐릭터 정보를 저장할 리스트 선언
    data_tuple_list = []
    
    # 모든 캐릭터 정보 추출
    character_info_list = html.select('table.rank_table > tbody > tr')
    
    # 정보 저장
    for character_info in character_info_list:
        nick = character_info.select_one('a').text
        job = character_info.select_one('dd').text
        union = character_info.select('td')[2].text.replace(',', '')
        img_url = character_info.select_one('span.char_img > img')["src"]
        data_tuple_list.append((nick, job, union, img_url))
        
    return data_tuple_list
